In [6]:
import pandas as pd
import numpy as np


df = pd.read_csv('original_data_by_municipality.csv', encoding='ISO-8859-1')


df.rename(columns={
    'Kanton (-) / Bezirk (>>) / Gemeinde (......)': 'Gemeinde',
    'Ja in %': 'Ja',
    'Datum und Vorlage': 'Datum und Vorlage'
}, inplace=True)


df['Kanton'] = np.nan
df['Bezirk'] = np.nan

current_kanton = "Test Kanton"
current_bezirk = "Test Bezirk"

for index, row in df.iterrows():
    if row['Gemeinde'].startswith('-'):
        current_kanton = row['Gemeinde'][1:].strip()
    elif row['Gemeinde'].startswith('>>'):
        current_bezirk = row['Gemeinde'][2:].strip()
    elif row['Gemeinde'].startswith('......'):
        
        df.at[index, 'Kanton'] = current_kanton
        df.at[index, 'Bezirk'] = current_bezirk

# Remove all rows that are not municipalities
df = df[df['Gemeinde'].str.startswith('......')]

# Remove the "......" from the Gemeinde column
df['Gemeinde'] = df['Gemeinde'].str[6:].str.strip()

# Convert the "Ja" column to a float
df['Ja'] = pd.to_numeric(df['Ja'], errors='coerce')
df['Ja'] = df['Ja'] / 100

# Pivot the df such that between "Gemeinde" and "Vorlage" is the "Ja" 
pivot_ja = df.pivot_table(index=['Gemeinde', 'Kanton', 'Bezirk'], 
                                   columns='Datum und Vorlage', 
                                   values='Ja')


pivot_ja.to_csv('cleaned_data_by_municipality.csv', index=False)

